In [ ]:
from math import log2

from z3 import *

In [ ]:
BitVec64 = lambda n: BitVec(n, bv=64)
BitVecVal64 = lambda v: BitVecVal(v, bv=64)

Range tracking part of value tracking will be done with the following C structure

```c
struct wrange {
	u64 start;
	u64 length;
}
```

In [ ]:
class Wrange:
    SIZE = 64 # Working with 64-bit integers
    name: str
    start: BitVecRef
    length: BitVecRef

    def __init__(self, name, start=None, length=None):
        self.name = name
        self.start = BitVec(f'Wrange-{name}-start', bv=self.SIZE) if start is None else start
        assert(self.start.size() == self.SIZE)
        self.length = BitVec(f'Wrange-{name}-length', bv=self.SIZE) if length is None else length
        assert(self.length.size() == self.SIZE)

    def print(self, model):
        name = self.name
        pad = ' ' * (len(self.name) + 1)
        start = model.eval(self.start).as_long()
        length = model.eval(self.length).as_long()
        end = model.eval(self.end).as_long()
        print(f'{name}(start={start}/{hex(start)},\n{pad}length={length}/{hex(length)},\n{pad}end={end}/{hex(end)})')

    def wellformed(self):
        return If(self.length == BitVecVal(2**64 - 1, bv=self.SIZE), self.start == 0, True)
        
    def reset(self):
        return And(self.start == BitVecVal(0, bv=self.SIZE), self.length == BitVecVal(-1, bv=self.SIZE))

    @property
    def end(self):
        return self.start + self.length
        
    @property
    def uwrapping(self):
        #return ULT(self.end, self.start)
        return ULT(BitVecVal(0, bv=self.SIZE) - self.start, self.length)
        
    @property
    def umin(self):
        return If(self.uwrapping, BitVecVal(0, bv=self.SIZE), self.start)

    @property
    def umax(self):
        end = self.start + self.length
        return If(self.uwrapping, BitVecVal(-1, bv=self.SIZE), end)
        
    def contains(self, val: BitVecRef):
        assert(val.size() == self.SIZE)
        nonwrapping_cond = And(ULE(self.umin, val), ULE(val, self.umax))
        wrapping_cond = Or(ULE(x, w1.end), ULE(w1.start, x))
        return If(self.uwrapping, wrapping_cond, nonwrapping_cond)

In [ ]:
x = BitVec64('x')
w1 = Wrange('w1', BitVecVal64(1), BitVecVal64(2))
prove(
    w1.contains(x) == Or(x == BitVecVal64(1), x == BitVecVal64(2), x == BitVecVal64(3))
)

x = BitVec64('x')
w1 = Wrange('w1', BitVecVal64(-1), BitVecVal64(2))
prove(
    w1.contains(x) == Or(x == BitVecVal64(-1), x == BitVecVal64(0), x == BitVecVal64(1))
)

## Addition

In [ ]:
def wrange_add(a: Wrange, b: Wrange):
    new_length = a.length + b.length
    too_wide = Or(ULT(new_length, a.length), ULT(new_length, b.length))
    new_start = If(too_wide, BitVecVal(0, a.SIZE), a.start + b.start)
    new_length = If(too_wide, BitVecVal(2**64-1, a.SIZE), a.length + b.length)
    return Wrange(f'{a.name} + {b.name}', If(new_length == BitVecVal(-1, bv=a.SIZE), BitVecVal(0, bv=a.SIZE), new_start), new_length)

In [ ]:
x = BitVec64('x')
w = wrange_add(
    # {1, 2, 3}
    Wrange('w1', BitVecVal64(1), BitVecVal64(2)),
    # + {0}
    Wrange('w2', BitVecVal64(0), BitVecVal64(0)),
)   # = {1, 2, 3}
prove(               # 1 <= x <= 3
    w.contains(x) == And(BitVecVal64(1) <= x, x <= BitVecVal64(3)),
)

x = BitVec64('x')
w = wrange_add(
    # {-1}
    Wrange('w1', BitVecVal64(-1), BitVecVal64(0)),
    # + {0, 1, 2}
    Wrange('w2', BitVecVal64(0), BitVecVal64(2)),   
)   # = {-1, 0, 1}
prove(               # -1 <= x <= 1
    w.contains(x) == And(BitVecVal64(-1) <= x, x <= BitVecVal64(1)),
)

In [ ]:
w1 = Wrange('w1')
w2 = Wrange('w2')
result = wrange_add(w1, w2)
x = BitVec64('x')
y = BitVec64('y')
premise = And(
    w1.wellformed(),
    w2.wellformed(),
    w1.contains(x),
    w2.contains(y),
)

In [ ]:
prove(
    Implies(
        premise,
        And(
            result.contains(x + y),
            result.wellformed(),
        ),
    )
)

## Arithmetic Negation

In [ ]:
def wrange_neg(a: Wrange):
    return Wrange(f'(-{a.name})', If(a.length == -1, 0, - a.start - a.length), a.length)

In [ ]:
x = BitVec64('x')
w = wrange_neg(
    # -{1, 2, 3}
    Wrange('w1', BitVecVal64(0x1), BitVecVal64(0x2)),
)   # = {-3, -2, -1}
prove(
    Implies(
        w.contains(x),
        And(-3 <= x, x <= -1),
    )
)

x = BitVec64('x')
w = wrange_neg(
    # -{-1}
    Wrange('w1', BitVecVal64(-1), BitVecVal64(0x0)),
)   # = { 1}
prove(
    Implies(
        w.contains(x),
        x == 1,
    )
)

In [ ]:
w1 = Wrange('w1')
result = wrange_neg(w1)
x = BitVec64('x')
premise = And(
    w1.wellformed(),
    w1.contains(x),
)

In [ ]:
prove(
    Implies(
        premise,
        And(
            result.contains(-x),
            result.wellformed(),
        ),
    )
)

## Subtraction

In [ ]:
def wrange_sub(a: Wrange, b: Wrange):
    # Be a bit lazy here, improve later
    w = wrange_add(a, wrange_neg(b))
    return Wrange(f'{a.name} - {b.name}', w.start, w.length)

In [ ]:
x = BitVec64('x')
w = wrange_sub(
    # {1, 2, 3}
    Wrange('w1', BitVecVal64(0x1), BitVecVal64(0x2)),
    # - {0}
    Wrange('w2', BitVecVal64(0x0), BitVecVal64(0x0)),
)   # = {1, 2, 3}
prove(
    Implies(
        w.contains(x),
        # 1 <= x <= 3
        And(ULE(1, x), ULE(x, 3)),
    )
)

x = BitVec64('x')
w = wrange_sub(
    # {-1}
    Wrange('w1', BitVecVal64(-1), BitVecVal64(0x0)),
    # - {0, 1, 2}
    Wrange('w2', BitVecVal64(0x0), BitVecVal64(0x2)),
)   # = {-3, -2, -1}
prove(
    Implies(
        w.contains(x),
        # -3 <= x <= -1
        And(-3 <= x, x <= -1),
    )
)

In [ ]:
w1 = Wrange('w1')
w2 = Wrange('w2')
result = wrange_sub(w1, w2)
x = BitVec64('x')
y = BitVec64('y')
premise = And(
    w1.wellformed(),
    w2.wellformed(),
    w1.contains(x),
    w2.contains(y),
)

In [ ]:
prove(
    Implies(
        premise,
        And(
            result.contains(x - y),
            result.wellformed(),
        ),
    )
)

## Multiplication

In [ ]:
def wrange_mul(a: Wrange, b: Wrange):
    too_wide = Or(UGT(a.length, BitVecVal(0xffffffff, bv=a.SIZE)), UGT(b.length, BitVecVal(0xffffffff, bv=b.SIZE)))
    new_start = If(too_wide, BitVecVal(0, a.SIZE), a.start * b.start)
    new_length = If(too_wide, BitVecVal(-1, a.SIZE), a.length * b.length)
    return Wrange(f'{a.name} * {b.name}', new_start, new_length)

In [ ]:
x = BitVec64('x')
w = wrange_mul(
    # {1, 2, 3}
    Wrange('w1', BitVecVal64(0x1), BitVecVal64(0x2)),
    # * {0}
    Wrange('w2', BitVecVal64(0x0), BitVecVal64(0x0)),
)   # = {0}
prove(
    Implies(
        w.contains(x),
        x == 0,
    )
)

x = BitVec64('x')
w = wrange_mul(
    # {-1}
    Wrange('w1', BitVecVal64(-1), BitVecVal64(0x0)),
    # {0, 1, 2}
    Wrange('w2', BitVecVal64(0x0), BitVecVal64(0x2)),
)   # {-2, -1, 0}
prove(
    Implies(
        w.contains(x),
        # -2 <= x <= 0
        And(-2 <= x, x <= 0),
    )
)

In [ ]:
w1 = Wrange('w1')
w2 = Wrange('w2')
result = wrange_mul(w1, w2)
x = BitVec64('x')
y = BitVec64('y')
premise = And(
    w1.wellformed(),
    w2.wellformed(),
    w1.contains(x),
    w2.contains(y),
)

In [ ]:
prove(
    Implies(
        premise,
        And(
            result.contains(x * y),
            result.wellformed(),
        ),
    )
)

## Evaluation

In [ ]:
s = Optimize()
x = BitVec64('x')
y = BitVec64('y')
w1 = Wrange('w1')
w2 = Wrange('w2')
result = wrange_add(w1, w2)
premise = And(
    w1.wellformed(),
    w2.wellformed(),
    w1.contains(x),
    w2.contains(y),
    x == BitVecVal64(-1),
)
s.minimize(y)
s.minimize(w1.length)
s.minimize(w2.start)
s.add(Not(
    Implies(
        premise,
        And(
            result.contains(x + y),
            result.wellformed(),
        ),
    )
))
s.check()

In [ ]:
m = s.model()
m

In [ ]:
f'x={m.eval(x)}, w1.contains(x)={m.eval(w1.contains(x))}'

In [ ]:
w1.print(m)

In [ ]:
f'y={m.eval(y)}, w2.contains(y)={m.eval(w2.contains(y))}'

In [ ]:
w2.print(m)

In [ ]:
result.print(m)

In [ ]:
f'x+y={m.eval(x+y)}, result.contains(x+y)={m.eval(result.contains(x+y))}'

In [ ]:
m.eval(w1.wellformed()), m.eval(w2.wellformed()), m.eval(result.wellformed())

In [ ]:
# wrapping?
m.eval(w1.uwrapping)

In [ ]:
# non-wrapping cond AND
m.eval(ULE(w1.start, x)), m.eval(ULE(x, w1.end))

In [ ]:
# wrapping cond OR
m.eval(ULE(x, w1.end)), m.eval(ULE(w1.start, x))